<a href="https://colab.research.google.com/github/younesabdolmalaky/Car-price/blob/main/Home_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c house-prices-advanced-regression-techniques

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 31.0MB/s]


In [2]:
! unzip /content/house-prices-advanced-regression-techniques.zip

Archive:  /content/house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder , MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
df = pd.read_csv('train.csv')
df

In [32]:
pd.options.display.max_rows = 10
(df.isnull().sum())

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [33]:
df = df.drop(['Id','MiscFeature' , 'Fence' , 'PoolQC' ,'Alley' , 'FireplaceQu'],axis = 'columns')

In [34]:
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(exclude=['float64' , 'int64']).columns

In [35]:
df = df.dropna(subset = categorical_cols)

In [36]:
df = pd.get_dummies(df)

In [37]:
imputer = KNNImputer(n_neighbors=3)
df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

In [38]:
df.isnull().sum().sum()

0

In [39]:
df['Price'] = df['SalePrice']
df = df.drop("SalePrice", axis='columns')

In [41]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

In [ ]:
df

In [51]:
y = data[: ,df.columns.get_loc("Price") ]
X = np.delete(data, df.columns.get_loc("Price") , axis=1)
y=y.astype('float')

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
!pip install cupy-cuda92 && \
!pip install pynvrtc && \
!/tmp/clean-layer.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 15.0 MB/s eta 0:00:00
/bin/bash: !pip: command not found


In [19]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)

# model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1,tree_method='gpu_hist', gpu_id=0)


param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500, 1000],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'colsample_bylevel': [0.5, 0.8, 1.0],
    'min_child_weight': [1, 2, 3, 4, 5],
    'reg_alpha': [0.01, 0.1, 1.0, 10.0],
    'reg_lambda': [0.01, 0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error' , verbose = 5)

grid_search.fit(X, y)

print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

best_model = grid_search.best_estimator_
best_model.fit(X, y)

Fitting 5 folds for each of 276480 candidates, totalling 1382400 fits
[CV 1/5] END colsample_bylevel=0.5, colsample_bytree=0.5, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.01, subsample=0.5;, score=-0.014 total time=   1.8s
[CV 2/5] END colsample_bylevel=0.5, colsample_bytree=0.5, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.01, subsample=0.5;, score=-0.015 total time=   0.6s
[CV 3/5] END colsample_bylevel=0.5, colsample_bytree=0.5, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.01, subsample=0.5;, score=-0.016 total time=   0.6s
[CV 4/5] END colsample_bylevel=0.5, colsample_bytree=0.5, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.01, subsample=0.5;, score=-0.017 total time=   0.6s
[CV 5/5] END colsample_bylevel=0.5, colsample_bytree=0.5, learning_rate=0.01, max_depth=3,

KeyboardInterrupt: ignored

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
import time

In [110]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=100,
                          task_type="GPU",
                          learning_rate=1,
                          depth=2, 
                          eval_metric='r2_score'
                          )
model.fit(X_train , y_train)
y_pred = model.predict(X_test)

CatBoostError: ignored

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=100,
                          task_type="GPU",
                          learning_rate=1,
                          depth=2, 
                          eval_metric='r2_score'
                          )
model.fit(X_train , y_train)
y_pred = model.predict(X_test)

CatBoostError: ignored

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=100,
                          task_type="GPU",
                          learning_rate=1,
                          depth=2, 
                          eval_metric='r2_score'
                          )
model.fit(X_train , y_train)
y_pred = model.predict(X_test)

CatBoostError: ignored

In [118]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=100000,
                          task_type="GPU",
                          learning_rate=0.01,
                          eval_metric='R2'
                          )
model.fit(X_train , y_train , eval_set = (X_test , y_test))
y_pred = model.predict(X_test)

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Streaming output truncated to the last 5000 lines.
67575:	learn: 0.9927226	test: 0.8765968	best: 0.8767146 (43570)	total: 12m 13s	remaining: 5m 51s
67576:	total: 12m 13s	remaining: 5m 51s
67577:	total: 12m 13s	remaining: 5m 51s
67578:	total: 12m 13s	remaining: 5m 51s
67579:	total: 12m 13s	remaining: 5m 51s
67580:	learn: 0.9927228	test: 0.8765948	best: 0.8767146 (43570)	total: 12m 13s	remaining: 5m 51s
67581:	total: 12m 13s	remaining: 5m 51s
67582:	total: 12m 13s	remaining: 5m 51s
67583:	total: 12m 13s	remaining: 5m 51s
67584:	total: 12m 13s	remaining: 5m 51s
67585:	learn: 0.9927229	test: 0.8765938	best: 0.8767146 (43570)	total: 12m 13s	remaining: 5m 51s
67586:	total: 12m 13s	remaining: 5m 51s
67587:	total: 12m 13s	remaining: 5m 51s
67588:	total: 12m 13s	remaining: 5m 51s
67589:	total: 12m 13s	remaining: 5m 51s
67590:	learn: 0.9927232	test: 0.8765916	best: 0.8767146 (43570)	total: 12m 13s	remaining: 5m 51s
67591:	total: 12m 13s	remaining: 5m 51s
67592:	total: 12m 13s	remaining: 5m 51s
6

KeyboardInterrupt: ignored

In [19]:
from xgboost.sklearn import XGBRegressor
best_model = XGBRegressor()
# best_model.fit(X_train , y_train)
# y_pred = best_model.predict(X_test)

In [30]:
import umap.umap_ as umap
reducer = umap.UMAP(n_components = 10)
embedding = reducer.fit_transform(X)


In [31]:
X_train, X_test, y_train, y_test = train_test_split(embedding, y, test_size=0.2, random_state=42)

In [32]:
from xgboost.sklearn import XGBRegressor
best_model = XGBRegressor()
best_model.fit(X_train , y_train)
y_pred = best_model.predict(X_test)

[15:59:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [29]:
r2_score(y_pred , y_test)

0.6026230050497328

In [ ]:
data_test_true = pd.DataFrame(X_test)
data_test_pred = pd.DataFrame(X_test)

In [ ]:
data_test_true['Price'] = y_test
data_test_pred['Price'] = y_pred

In [ ]:
inverserd_data_true = scaler.inverse_transform(data_test_true)
inverserd_data_pred = scaler.inverse_transform(data_test_pred)

In [ ]:
pd.Dataframe(inverserd_data_true)

In [ ]:
y_test = inverserd_data_true[:,265]
y_pred = inverserd_data_pred[:,265]

In [ ]:
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))

In [ ]:
r2_score(y_test ,y_pred)

In [ ]:
#importing the necessary libraries 
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, confusion_matrix 
  
#loading the dataset 
data = pd.read_csv('dataset.csv') 
  
#separating the independent and dependent variables 
X = data.iloc[:, :-1].values  #independent variables 
y = data.iloc[:, -1].values   #dependent variable (target)  
  
#handling missing values with mean imputation  
from sklearn.impute import SimpleImputer    #for missing values in numerical columns  
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')    #creating an object of SimpleImputer class    #fitting the imputer object to our dataset  
imputer = imputer.fit(X[:, :])     #replacing missing values with mean of respective column    X[:, :] = imputer.transform(X[:, :])     #handling categorical data with label encoding    le = LabelEncoder()     #converting all categorical columns into numerical columns    X[:, 4] = le.fit_transform(X[:, 4])      #one hot encoding for multi-categorical columns    ohe = OneHotEncoder(categorical_features=[4])     X = ohe.fit_transform(X).toarray()      #splitting the dataset into training and testing sets    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)      #building a random forest classifier model    model = RandomForestClassifier()     model.fit(X_train, y_train)      #predicting on test set and calculating accuracy score    ypreds = model.predict(X_test)     print("Accuracy Score:", accuracy_score(ytest, ypreds))



In [ ]:
import pandas as pd 
import numpy as np 

#read in the dataset 
df = pd.read_csv('my_dataset.csv') 

#create new features by combining existing columns 
df['new_feature1'] = df['col1'] + df['col2'] 
df['new_feature2'] = df['col3'] * df['col4'] 
df['new_feature3'] = np.sqrt(df['col5']) 
df['new_feature4'] = np.log(df['col6']) 

#save the new dataset with the new features included 
df.to_csv('my_dataset_with_features.csv', index=False)